<a href="https://colab.research.google.com/github/jingw222/twitterbot-gpt2/blob/master/gpt_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-tuning custom datasets on [GPT-2-small](https://github.com/openai/gpt-2)

In [0]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
!git clone https://github.com/jingw222/twitterbot-gpt2.git

Cloning into 'twitterbot-gpt2'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 4), reused 24 (delta 3), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [7]:
!pip3 install -r requirements.txt

    100% |████████████████████████████████| 604kB 26.9MB/s 
    100% |████████████████████████████████| 61kB 28.6MB/s 
    100% |████████████████████████████████| 51kB 24.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
spacy 2.0.18 has requirement regex==2018.01.10, but you'll have regex 2017.4.5 which is incompatible.
google-colab 1.0.0 has requirement requests~=2.18.0, but you'll have requests 2.21.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: regex 2018.1.10
    Uninstalling regex-2018.1.10:
      Successfully uninstalled regex-2018.1.10
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4
  Found ex

Mount your Google Drive storage for convenient backup and restore.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r /content/drive/My\ Drive/Projects/twitterbot/* /content/

In [0]:
sys.path.append('/content/twitterbot-gpt2/src')

In [5]:
cd twitterbot-gpt2

/content/twitterbot-gpt2


In [6]:
!mkdir data
!wget -P data/ https://www.researchgate.net/profile/Kasra_Madadipouya/publication/304742521_CSV_dataset_of_76000_quotes_suitable_for_quotes_recommender_systems_or_other_analysis/data/5778c4ac08ae4645d611b10d/quotes-all.csv

--2019-04-22 13:57:38--  https://www.researchgate.net/profile/Kasra_Madadipouya/publication/304742521_CSV_dataset_of_76000_quotes_suitable_for_quotes_recommender_systems_or_other_analysis/data/5778c4ac08ae4645d611b10d/quotes-all.csv
Resolving www.researchgate.net (www.researchgate.net)... 104.17.33.105, 104.17.32.105
Connecting to www.researchgate.net (www.researchgate.net)|104.17.33.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12545341 (12M) [text/plain]
Saving to: ‘data/quotes-all.csv’

quotes-all.csv      100%[===================>]  11.96M  27.1MB/s    in 0.4s    

2019-04-22 13:57:39 (27.1 MB/s) - ‘data/quotes-all.csv’ saved [12545341/12545341]



In [8]:
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 538kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 54.4Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 906kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:06, 71.3Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 3.49Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 49.8Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 47.8Mit/s]                                                       


In [0]:
df = pd.read_csv(os.path.join('data', 'quotes-all.csv'), sep=';', header=None, names=['quotes', 'auther', 'category'])

In [13]:
df.head()

,quotes,auther,category
0,Age is an issue of mind over matter. If you do...,Mark Twain,age
1,"Anyone who stops learning is old, whether at t...",Henry Ford,age
2,Wrinkles should merely indicate where smiles h...,Mark Twain,age
3,True terror is to wake up one morning and disc...,Kurt Vonnegut,age
4,A diplomat is a man who always remembers a wom...,Robert Frost,age


In [11]:
df.shape

(75966, 3)

In [0]:
# Output column `quotes` to txt 
df['quotes'].to_csv(os.path.join('data', 'quotes-all.txt'), sep='\t', header=None, index=None)

Encode the training data.

In [15]:
!PYTHONPATH=src ./encode.py data/quotes-all.txt data/quotes-all.npz

Reading files
100% 1/1 [00:13<00:00, 13.67s/it]
Writing data/quotes-all.npz


Start fine-tuning the pre-trained gpt-2 model.

In [59]:
!PYTHONPATH=src ./train.py --dataset data/quotes-all.npz --batch_size 2 --save_every 2000

2019-04-21 05:26:36.199637: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-21 05:26:36.199897: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x189c260 executing computations on platform Host. Devices:
2019-04-21 05:26:36.199934: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-21 05:26:36.384326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-21 05:26:36.384842: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x189be40 executing computations on platform CUDA. Devices:
2019-04-21 05:26:36.384900: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-04-21 05:26:36.385258: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

Move the fine-tuned model checkpoint to the designated model directory.

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

Make a directory snapshot to Google Drive.

In [67]:
!zip -r /content/twitterbot-gpt2/checkpoint/run1/models.zip /content/gpt-2/checkpoint/run1/*

  adding: content/gpt-2/checkpoint/run1/checkpoint (deflated 64%)
  adding: content/gpt-2/checkpoint/run1/counter (stored 0%)
  adding: content/gpt-2/checkpoint/run1/events.out.tfevents.1555824429.d0a649aba86d (deflated 60%)
  adding: content/gpt-2/checkpoint/run1/model-4339.data-00000-of-00001 (deflated 7%)
  adding: content/gpt-2/checkpoint/run1/model-4339.index (deflated 62%)
  adding: content/gpt-2/checkpoint/run1/model-4339.meta (deflated 91%)


In [0]:
!cp -r /content/twitterbot-gpt2/ /content/drive/My\ Drive/Projects/twitterbot-gpt2/

And finally, we can play with the model by generating some samples, either interactively or unconditionally.

In [64]:
!python3 src/interactive_conditional_samples.py --top_k 40 --temperature 0.5

2019-04-21 11:17:08.575737: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-21 11:17:08.575985: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x3560260 executing computations on platform Host. Devices:
2019-04-21 11:17:08.576029: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-21 11:17:08.742259: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-21 11:17:08.742776: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x355fe40 executing computations on platform CUDA. Devices:
2019-04-21 11:17:08.742806: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-04-21 11:17:08.743183: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

In [66]:
!python3 src/generate_unconditional_samples.py --temperature 1.0 --top_k 40 --length 500

2019-04-21 11:28:54.494893: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-21 11:28:54.495148: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x16cc260 executing computations on platform Host. Devices:
2019-04-21 11:28:54.495180: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-21 11:28:54.661634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-21 11:28:54.662192: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x16cbe40 executing computations on platform CUDA. Devices:
2019-04-21 11:28:54.662225: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-04-21 11:28:54.662555: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de